In [88]:
#!/usr/bin/python3
import argparse
from random import uniform, randint

def generate_problem(
    variables=10, 
    constraint_probability=0.25, 
    R=100,
    min_intervals=1, 
    max_intervals=5,
    scaling_factor=1,
    ):
    
    T_all = []
    
    for j in range(variables):
        T_all.append({
            'i': 0,
            'j': j,
            'intervals': [(-R, R)],
        })
        
    for i in range(1, variables):
        for j in range(i+1, variables):
            if uniform(0, 1) < constraint_probability:
                num_intervals = randint(min_intervals, max_intervals)
                points = [randint(-R, R) for _ in range(2 * num_intervals)]
                points.sort()
                
                intervals_temp = list(zip(points[::2], points[1::2]))
                
                # apply scaling factor:
                intervals = []
                for a, b in intervals_temp:
                    midpoint = int((a+b)/2)
                    l = int( midpoint - (midpoint-a)*scaling_factor )
                    r = int( midpoint + (b-midpoint)*scaling_factor )
                    intervals.append( (l, r) )
                    
                T_all.append({
                    'i': i,
                    'j': j,
                    'intervals': intervals,
                })

    return T_all

#!/usr/bin/python3
import argparse
from random import uniform, randint

def generate_problem(
    variables=10, 
    constraint_probability=0.25, 
    R=100,
    min_intervals=1, 
    max_intervals=5,
    scaling_factor=1,
    ):
    
    T_all = []
    
    for j in range(variables):
        T_all.append({
            'i': 0,
            'j': j,
            'intervals': [(-R, R)],
        })
        
    for i in range(1, variables):
        for j in range(i+1, variables):
            if uniform(0, 1) < constraint_probability:
                num_intervals = randint(min_intervals, max_intervals)
                points = [randint(-R, R) for _ in range(2 * num_intervals)]
                points.sort()
                
                intervals_temp = list(zip(points[::2], points[1::2]))
                
                # apply scaling factor:
                intervals = []
                for a, b in intervals_temp:
                    midpoint = int((a+b)/2)
                    l = int( midpoint - (midpoint-a)*scaling_factor )
                    r = int( midpoint + (b-midpoint)*scaling_factor )
                    intervals.append( (l, r) )
                    
                T_all.append({
                    'i': i,
                    'j': j,
                    'intervals': intervals,
                })

    return T_all

def verify_witness(X, T):
    failed_constraints = []
    for constraint in T:
        i = constraint['i']
        j = constraint['j']

        diff = X[j] - X[i]

        passed = False
        for interval in constraint['intervals']:
            if interval[0] <= diff <= interval[1]:
                passed = True
                break
        
        if not passed:
            failed_constraints += [f"X_i={X[i]}, X_j={X[j]} on {constraint}"]

    return failed_constraints

In [62]:
from pprint import pprint

In [236]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def fitness(gene, T):
    # TODO
    # different ways of computing fitness
    # like: max distance for constraint to be satisfied
    #       number of unsatisfied constraints
    #       this, but weighted somehow, etc
    
    return -len(verify_witness(gene, T))

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]

def crossover(genes, gene_pool_size):
    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, gene_pool_size-2)
        genes += [ genes[i][cross_index:] + genes[j][:cross_index] ]
        
    return genes

def mutate(genes, mutation_chance, r):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
            g[randint(0, len(g)-1)] = randint(-r, r)
            
    return genes
        
def genetic(T, r,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T, r) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size)
        genes = mutate(genes, mutation_chance, r)
    
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = verify_witness(best_gene, T)
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [240]:
T = generate_problem(20, scaling_factor=1)
genetic(
    T=T, r=100,
    gene_pool_size=50,
    retainment_ratio=0.1,
    mutation_chance=0.1,
    max_iterations=2000,
)

best gene: [-14, -9, -23, -83, -49, -10, -55, -47, -72, 24, 4, -29, 51, -15, -27, 66, -26, 49, -33, -2]
constraints failed: 4 out of: 60
failed constraints: ["X_i=-9, X_j=-47 on {'i': 1, 'j': 7, 'intervals': [(-12, 42)]}", "X_i=-72, X_j=4 on {'i': 8, 'j': 10, 'intervals': [(-56, -38), (-26, -21), (-20, -6), (2, 6), (62, 67)]}", "X_i=51, X_j=-33 on {'i': 12, 'j': 18, 'intervals': [(15, 95)]}", "X_i=-26, X_j=49 on {'i': 16, 'j': 17, 'intervals': [(-77, -76), (-6, -1), (16, 34)]}"]
